In [63]:
import pandas as pd 
import numpy as np
import seaborn as sns
import sklearn as sk
import matplotlib.pyplot as plt
import json
from pandas import json_normalize
from dateutil import parser
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [45]:
"This is to load the games database onto a pandas dataframe, which allows us to manage the data more easily"
games=pd.read_json("output_steam_games.json",lines=True)

In [ ]:
"Here, we  will check the head of the games database to see if the information is correct"
games.info()
gameswna=games.dropna(axis='index',how='all')
gameswna.reset_index(inplace=True)

In [ ]:
gameswna.dropna(axis=0,how='any',inplace=True)
gameswna.drop(['publisher','index','app_name','url','tags','reviews_url','specs','price','early_access'],axis=1,inplace=True)

In [ ]:
gameswna.head()

In [ ]:
gameswna['genres']=gameswna['genres'].apply(lambda x:', '.join(x))
gameswna['release_year'] = gameswna['release_date'].str.extract(r'(\d{4})', expand=False)

In [50]:
gameswna_dummies = gameswna['genres'].str.get_dummies(', ')
gameswna = pd.concat([gameswna, gameswna_dummies], axis=1)
gameswna.drop(['genres','release_date'],axis=1,inplace=True)

In [52]:
gameswna.to_csv('clean_output_steam_games.csv',index=False)

In [58]:
"This step is done to load the reviews database onto a pandas dataframe, which allows us to manage the data in a more efficient way"
with open("australian_user_reviews.json",'r', encoding ='utf-8') as rw:
    data=rw.readlines()
db=[eval(line.strip()) for line in data]
reviews=pd.DataFrame(db)

In [59]:
"Here, we  will check the head of the reviews database to see if the information is correct"
reviews.info()
reviewswna=reviews.dropna(axis='index',how='all')
reviewswna.reset_index(inplace=True)
reviewswna.drop(['user_url','index'],axis=1,inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [60]:
reviewswna.head()

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."


In [61]:
# Since the reviews column database has more than one review, we use the explode function in order to separate every review done by the user and we store it into a new table.
df_exploded = reviewswna.explode('reviews')
# Afterwards, we reset the index in order to match the amount of columns, which is 59333.
df_exploded.reset_index(inplace=True)
# By using the drop index, we make sure to reset every single index in order for it to show us the entirety of the 59333 indexes by getting rid of the old indexes.
df_exploded.drop('index',axis=1,inplace=True)
# On the other hand, by using the json_normalize function, we make sure to separate the fields on the reviews column, this for us to get rid of the information we don't need or won't be using.
df_exploded1=json_normalize(reviewswna['reviews'].explode())

In [ ]:
# Here, we check the amount of rows of the new table we created.
#df_exploded #59333 rows
# Likewise
#df_exploded1 #59333 rows
# With this line we make sure to merge the split columns of the review section and the split rows of the user_id so there are no null users when merging.
reviewswna1=pd.concat([df_exploded,df_exploded1],axis=1)
# We proceed to get rid of the columns we wont be using towards the objective of the project and also remove any Nas that were created in the process, and also reset the indexes.
reviewswna1.drop(['reviews','helpful','last_edited','funny'], axis=1, inplace=True)
reviewswna1.dropna(inplace=True)
reviewswna1.reset_index(inplace=True)
reviewswna1.drop('index',axis=1,inplace=True)

In [64]:
# By using dateutil.parser, we create a function in order to extract the year from the posted column
def extract_year(date_string):
    try:
        # We try to analyze the date with various formats
        parsed_date = parser.parse(date_string)
        return parsed_date.year
    except ValueError:
        # If this doesn't work, we try another format.
        try:
            parsed_date = parser.parse(date_string, fuzzy=True)
            return parsed_date.year
        except ValueError:
            # If it fails again, it just returns NaT(not a time)
            return pd.NaT

In [65]:
# By applying the function we just created, we add a new column to our dataframe called year
reviewswna1['year']=reviewswna1['posted'].apply(extract_year)
# By using the unique function, we evaluate outliers to see why are they being created, in this case, we identify that there's an outlier with the year 2024, when we check the database,
# we discover that the rows that have the 2024 didn't have a year on the posted column, so we proceed to delete the rows that contain the 2024 because that number was assigned by default since the field didn't have a year.
reviewswna1['year'].unique()|

In [87]:
# We apply a filter to get rid of the 2024 values, afterwards, we proceed to reset the indexes and finally, finish cleaning the table.
reviewswna2f=reviewswna2[reviewswna2['year']!=2024]
reviewswna2f.reset_index(inplace=True)
reviewswna2f.drop('index',axis=1,inplace=True)
reviewswna2f.drop('posted',axis=1,inplace=True)
reviewswna2f['item_id']=reviewswna2f['item_id'].astype(int)
reviewswna2f['year']=reviewswna2f['year'].astype(int)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_15776\2250573296.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewswna2f.drop('index',axis=1,inplace=True)


In [159]:
# Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Function to assign values according to the scale
def get_sentiment_score(text):
    if pd.isnull(text) or text == '':
        return 1  # Return neutral if it is empty or NaN
    elif isinstance(text, str):
        sentiment = sia.polarity_scores(text)
        compound_score = sentiment['compound']
        if compound_score >= -0.05:
            return 2  # Good score
        elif compound_score <= -0.05:
            return 0  # Bad score
        else:
            return 1
    else:
        return 1  # Return neutral for non-string values

In [ ]:
reviewswna2f.drop('sentiment_analysis',axis=1,inplace=True)
reviewswna2f['sentiment_analysis']=reviewswna2f['review'].apply(get_sentiment_score)

In [162]:
# We check our dataframe to see if it does have the information we need before saving it to a csv
reviewswna2f
# We save our dataframe into a csv database.
reviewswna2f.to_csv('clean_output_user_reviews.csv',index=False)

In [93]:
"This step is done to load the items database onto a pandas dataframe, which allows us to manage the data in a more efficient way"
with open("australian_users_items.json",'r',encoding='utf-8') as it:
    data=it.readlines()
db1=[eval(line.strip()) for line in data]
items=pd.DataFrame(db1)

In [94]:
"Here, we  will check the head of the items database to see if the information is correct"
items.info()
itemswna=items.dropna(axis='index',how='all')
itemswna.reset_index(inplace=True)
#Here we are deleting from our dataframe the columns that we don't need, in this case, we reset the indexes if there's a change on them after deleting the duplicates and Na's
itemswna.drop(['index','steam_id','user_url','items_count'],axis=1,inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [95]:
itemswna.head()

,user_id,items
0,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [97]:
itemsf=itemswna.explode('items')
itemsf.reset_index(inplace=True)
itemsf.drop('index',axis=1,inplace=True)
itemsf2=json_normalize(itemswna['items'].explode())
itemswnaf=pd.concat([itemsf,itemsf2],axis=1)
itemswnaf.drop(['items','playtime_2weeks'],axis=1,inplace=True)
itemswnaf.dropna(axis=0,how='any',inplace=True)
itemswnaf.reset_index(inplace=True)
itemswnaf.drop('index',axis=1,inplace=True)

In [147]:
#itemsf #5170015 rows
#itemsf2 #5170015 rows
itemswnaf#5153209 rows
itemswnaf.to_csv('clean_output_user_items.csv',index=False)

In [53]:
# Since we already have the information stored as CSV format, we will proceed by reading them and saving them into pandas dataframes.
items=pd.read_csv('clean_output_user_items.csv')
reviews=pd.read_csv('clean_output_user_reviews.csv')
games=pd.read_csv('clean_output_steam_games.csv')

In [56]:
# We will check the first 5 rows of each database in order to see the column that will be used to merge both databases.
games.head()
items.head()

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6.0
1,76561197970982479,20,Team Fortress Classic,0.0
2,76561197970982479,30,Day of Defeat,7.0
3,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0


In [57]:
# With this code lines what we're doing is merging the items database and the games database, in order to have the information we need in one database.
# We also clean any na's that may appear, reset the indexes and drop the columns that we no longer need or are duplicated, finally we store the new merged database into a CSV file.
maindf=pd.merge(items,games,how='outer',left_on='item_id',right_on='id')
maindf1=maindf.dropna(axis=0,how='any')
maindf1.reset_index(inplace=True)
maindf1.drop(['item_id','item_name','index'],axis=1,inplace=True)
maindf1.to_csv('maindb.csv',index=False)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_8640\1446749925.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maindf1.drop(['item_id','item_name','index'],axis=1,inplace=True)


In [ ]:
alldb=pd.read_csv('maindb.csv')
alldbgames=alldb.drop_duplicates(subset='title')
alldbgames.drop(['release_year','user_id','playtime_forever','Action','Adventure','Audio Production','Early Access','Photo Editing','RPG','Racing','Simulation','Software Training','Sports','Web Publishing','Utilities','Strategy','Video Production','Casual','Animation &amp; Modeling','Design &amp; Illustration','Education','Free to Play','Indie','Design &amp; Illustration','Massively Multiplayer'],axis=1,inplace=True)
alldbgames.reset_index(inplace=True)
alldbgames.drop('index',axis=True,inplace=True)
alldbgames.to_csv('steam_games_devel.csv')

In [ ]:
maindb1=alldb.head(300000)
maindb1.to_csv('maindb.csv')